# Langchain Intro

In [1]:
# Built-in library
import re
import json
from typing import Any, Optional, Union
import logging
import warnings

# Standard imports
import numpy as np
from pathlib import Path
import pandas as pd
import polars as pl
from pprint import pprint
from rich import print
import torch

# Visualization
import matplotlib.pyplot as plt


# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv, find_dotenv

from langchain.llms import OpenAI

### Using A Basic OpenAI Langchain LLM

- Without using a chain.
- Very simplistic.

<br>

[![image.png](https://i.postimg.cc/cLyVwqrs/image.png)](https://postimg.cc/Mv5PJF74)


In [3]:
_ = load_dotenv(find_dotenv())  # read local .env file
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_GPT_MODEL: str = "gpt-3.5-turbo-instruct"
TEMPERATURE: float = 0.5

In [4]:
# Create LLM
llm: OpenAI = OpenAI(
    openai_api_key=OPENAI_API_KEY,
    model=OPENAI_GPT_MODEL,
    temperature=TEMPERATURE,
)
prompt: str = "Write a very short poem."
result = llm(prompt=prompt)

print(result)

"Sun shines bright,
Birds take flight,
Nature's delight,
Pure and light."

## Updated Logic

- Add Chains (from langchain)

<br>

[![image.png](https://i.postimg.cc/RVLp2WgH/image.png)](https://postimg.cc/0brCwyRk)

In [5]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

### LLM Chain

[![image.png](https://i.postimg.cc/05VBzdbB/image.png)](https://postimg.cc/0rJf3SGY)

In [6]:
# Used for text completion LLMs
prompt_template: str = """Write a very short {language} function that will {task}"""
code_prompt: PromptTemplate = PromptTemplate(
    input_variables=["language", "task"], template=prompt_template
)
code_chain: LLMChain = LLMChain(llm=llm, prompt=code_prompt)

# Hardcoded inputs!
language, task = ("Python", "generate a list of numbers")

result = code_chain(inputs={"language": language, "task": task})
print(result)

{
    'language': 'Python',
    'task': 'generate a list of numbers',
    'text': '\n\ndef generate_list(n):\n    return list(range(n))'
}

In [7]:
print(result.get("text"))

def generate_list(n):
    return list(range(n))

## Connecting Chains Together

- Chains in series (sequential chain)

<br>

[![image.png](https://i.postimg.cc/8Cdgm9wV/image.png)](https://postimg.cc/GBpfcMZM)

In [8]:
from langchain.chains import SequentialChain

# Used for text completion LLMs
code_prompt_template: str = (
    """Write a very short {language} function that will {task}"""
)
unittest_prompt_template: str = (
    """Write a unit test for the following {language} code: \n{code}"""
)

# Propmt(s)
code_prompt: PromptTemplate = PromptTemplate(
    input_variables=["language", "task"], template=code_prompt_template
)
unittest_prompt: PromptTemplate = PromptTemplate(
    input_variables=["language", "code"], template=unittest_prompt_template
)

# Chain(s)
code_chain: LLMChain = LLMChain(
    llm=llm,
    prompt=code_prompt,
    output_key="code",  # from text to code
)
unittest_chain: LLMChain = LLMChain(
    llm=llm,
    prompt=unittest_prompt,
    output_key="test",  # from text to test
)
# Feed the output of code_chain into unittest_chain
final_chain: SequentialChain = SequentialChain(
    chains=[code_chain, unittest_chain],
    input_variables=["language", "task"],
    output_variables=["code", "test"],
)
# Hardcoded inputs!
language, task = ("Python", "generate a list of numbers")

result = final_chain(inputs={"language": language, "task": task})
print(result)

{
    'language': 'Python',
    'task': 'generate a list of numbers',
    'code': '\n\ndef generate_list(start, end):\n    return list(range(start, end+1))',
    'test': '\n\nimport unittest\n\nclass TestGenerateList(unittest.TestCase):\n    \n    def 
test_generate_list(self):\n        # Test list generation with positive numbers\n        
self.assertEqual(generate_list(1,5), [1,2,3,4,5])\n        \n        # Test list generation with negative numbers\n
self.assertEqual(generate_list(-5,0), [-5,-4,-3,-2,-1,0])\n        \n        # Test list generation with start and 
end being the same number\n        self.assertEqual(generate_list(1,1), [1])\n        \n        # Test list 
generation with start and end being negative numbers\n        self.assertEqual(generate_list(-10,-5), 
[-10,-9,-8,-7,-6,-5])\n        \n        # Test list generation with start and end being decimal numbers\n        
self.assertEqual(generate_list(1.5,5.5), [1.5,2.5,3.5,4.5,5.5])\n        \n        # Test list generation with 
start and end being non-integer numbers\n        self.assertEqual(generate_list(1.5,3.5), [1.5,2.5,3.5])\n        
\n        # Test list generation with start and end being'
}

In [9]:
# Output of the unittest_chain (key="test")
print(result.get("test"))

import unittest

class TestGenerateList(unittest.TestCase):
    
    def test_generate_list(self):
        # Test list generation with positive numbers
        self.assertEqual(generate_list(1,5), [1,2,3,4,5])
        
        # Test list generation with negative numbers
        self.assertEqual(generate_list(-5,0), [-5,-4,-3,-2,-1,0])
        
        # Test list generation with start and end being the same number
        self.assertEqual(generate_list(1,1), [1])
        
        # Test list generation with start and end being negative numbers
        self.assertEqual(generate_list(-10,-5), [-10,-9,-8,-7,-6,-5])
        
        # Test list generation with start and end being decimal numbers
        self.assertEqual(generate_list(1.5,5.5), [1.5,2.5,3.5,4.5,5.5])
        
        # Test list generation with start and end being non-integer numbers
        self.assertEqual(generate_list(1.5,3.5), [1.5,2.5,3.5])
        
        # Test list generation with start and end being

<hr><br>

# Simple Terminal Chatbot

<br>

### Chat Model

- It's optimized for conversations.
- It has the following inputs:
  - **System Message:** A system message is a message generated by the system or platform hosting the conversation. It is not authored by a human user or the AI model.
  
  - **Human Message:** Any message created by a human user that is directed towards the LLM for processing or interaction.

  - **AI/Assistant Message:** The output generated by the LLM in response to a human message or internal processing.
  
[![image.png](https://i.postimg.cc/FRgxXmJ6/image.png)](https://postimg.cc/YG0FQ5RN)

<br>

### ChatPrompt Template

[![image.png](https://i.postimg.cc/s2nTM32Z/image.png)](https://postimg.cc/4K96MkVJ)

In [17]:
OPENAI_CHAT_MODEL: str = "gpt-3.5-turbo"

In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import (
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    SystemMessagePromptTemplate,
    ChatPromptTemplate,
)

In [18]:
chat_llm = ChatOpenAI(
    model=OPENAI_CHAT_MODEL,
    temperature=TEMPERATURE,
)

# Used for chat completion LLMs
prompt = ChatPromptTemplate(
    input_variables=["content"],
    messages=[HumanMessagePromptTemplate.from_template("{content}")],
)

chain: LLMChain = LLMChain(llm=chat_llm, prompt=prompt)

In [19]:
content: str = "Hello, who are you?"

result: dict[str, Any] = chain({"content": content})
print(result)

{
    'content': 'Hello, who are you?',
    'text': 'Hello! I am an AI language model developed by OpenAI. I am designed to assist with answering questions
and engaging in conversations on a wide range of topics. How can I assist you today?'
}

### Add Memory

- Memory allows LLMs to access and consider previous conversations or inputs, leading to more coherent and contextually relevant responses.

<br>

[![image.png](https://i.postimg.cc/MGPkWySn/image.png)](https://postimg.cc/8F6X4fsG)

<br><br>

#### Message PlaceHolder

- It's used to track the conversations.
- It contains the contents of the chat. i.e. the human and the AI messages.

[![image.png](https://i.postimg.cc/3wPBCRjL/image.png)](https://postimg.cc/p9JzPP68)


In [20]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder

In [23]:
chat_llm = ChatOpenAI(
    model=OPENAI_CHAT_MODEL,
    temperature=TEMPERATURE,
)

memory = ConversationBufferMemory(
    memory_key="messages",  # key for storing the chat history
    return_messages=True,  # returns the promp classes. e.g. HumanMsg, AIMsg, etc
)

# Used for chat completion LLMs
prompt = ChatPromptTemplate(
    input_variables=["content", "messages"],
    messages=[
        HumanMessagePromptTemplate.from_template("{content}"),
        MessagesPlaceholder(variable_name="messages"), # used to track the conversations
    ],
)

chain: LLMChain = LLMChain(
    llm=chat_llm,
    prompt=prompt,
    memory=memory,  # Add memory (new!)
)

In [24]:
content: str = "Hello, who are you?"

result: dict[str, Any] = chain({"content": content})
print(result)

{
    'content': 'Hello, who are you?',
    'messages': [
        HumanMessage(content='Hello, who are you?', additional_kwargs={}, example=False),
        AIMessage(
            content='Hello! I am an AI language model developed by OpenAI. I am here to assist you with any 
questions or tasks you may have. How can I help you today?',
            additional_kwargs={},
            example=False
        )
    ],
    'text': 'Hello! I am an AI language model developed by OpenAI. I am here to assist you with any questions or 
tasks you may have. How can I help you today?'
}

In [25]:
# Follow up question
content: str = "What is NLP?"

result: dict[str, Any] = chain({"content": content})
print(result)

{
    'content': 'What is NLP?',
    'messages': [
        HumanMessage(content='Hello, who are you?', additional_kwargs={}, example=False),
        AIMessage(
            content='Hello! I am an AI language model developed by OpenAI. I am here to assist you with any 
questions or tasks you may have. How can I help you today?',
            additional_kwargs={},
            example=False
        ),
        HumanMessage(content='What is NLP?', additional_kwargs={}, example=False),
        AIMessage(
            content='NLP stands for Natural Language Processing. It is a field of artificial intelligence that 
focuses on the interaction between computers and human language. NLP involves the development of algorithms and 
models that enable computers to understand, interpret, and generate human language in a way that is meaningful and 
useful. It encompasses tasks such as text classification, sentiment analysis, language translation, speech 
recognition, and information extraction, among others. NLP plays a crucial role in various applications, including 
virtual assistants, chatbots, search engines, and language translation services.',
            additional_kwargs={},
            example=False
        )
    ],
    'text': 'NLP stands for Natural Language Processing. It is a field of artificial intelligence that focuses on 
the interaction between computers and human language. NLP involves the development of algorithms and models that 
enable computers to understand, interpret, and generate human language in a way that is meaningful and useful. It 
encompasses tasks such as text classification, sentiment analysis, language translation, speech recognition, and 
information extraction, among others. NLP plays a crucial role in various applications, including virtual 
assistants, chatbots, search engines, and language translation services.'
}

In [26]:
# Follow up question
content: str = "How can I study it?"

result: dict[str, Any] = chain({"content": content})
print(result)

{
    'content': 'How can I study it?',
    'messages': [
        HumanMessage(content='Hello, who are you?', additional_kwargs={}, example=False),
        AIMessage(
            content='Hello! I am an AI language model developed by OpenAI. I am here to assist you with any 
questions or tasks you may have. How can I help you today?',
            additional_kwargs={},
            example=False
        ),
        HumanMessage(content='What is NLP?', additional_kwargs={}, example=False),
        AIMessage(
            content='NLP stands for Natural Language Processing. It is a field of artificial intelligence that 
focuses on the interaction between computers and human language. NLP involves the development of algorithms and 
models that enable computers to understand, interpret, and generate human language in a way that is meaningful and 
useful. It encompasses tasks such as text classification, sentiment analysis, language translation, speech 
recognition, and information extraction, among others. NLP plays a crucial role in various applications, including 
virtual assistants, chatbots, search engines, and language translation services.',
            additional_kwargs={},
            example=False
        ),
        HumanMessage(content='How can I study it?', additional_kwargs={}, example=False),
        AIMessage(
            content="If you are interested in studying NLP, here are some steps you can take:\n\n1. Learn the 
basics: Start by understanding the fundamentals of natural language processing, including key concepts, techniques,
and algorithms. Familiarize yourself with topics such as text preprocessing, tokenization, part-of-speech tagging, 
named entity recognition, and syntactic parsing.\n\n2. Programming skills: Develop your programming skills, as NLP 
often involves working with large datasets and implementing algorithms. Python is a popular language for NLP, so 
learning Python and libraries like NLTK (Natural Language Toolkit) and spaCy can be beneficial.\n\n3. Study 
algorithms and models: Dive deeper into the algorithms and models used in NLP, such as Naive Bayes, Hidden Markov 
Models, Conditional Random Fields, and neural networks. Understand their principles, strengths, and 
limitations.\n\n4. Explore NLP libraries and tools: Familiarize yourself with popular NLP libraries and tools that 
provide pre-built functionalities for various NLP tasks. Some widely used libraries include NLTK, spaCy, 
TensorFlow, and PyTorch.\n\n5. Practice with datasets: Work with real-world datasets to gain hands-on experience. 
There are many publicly available datasets for tasks like sentiment analysis, text classification, and named entity
recognition. Practice preprocessing the data, training models, and evaluating their performance.\n\n6. Stay 
updated: NLP is a rapidly evolving field, so it's essential to stay updated with the latest research papers, 
conferences, and industry trends. Follow NLP blogs, attend conferences, and join online communities to stay 
connected with the NLP community.\n\n7. Build projects: Implement NLP projects to apply your knowledge and showcase
your skills. Choose projects that align with your interests, such as developing a chatbot, sentiment analysis 
system, or language translation model. Building projects will help solidify your understanding and demonstrate your
abilities to potential employers or collaborators.\n\n8. Collaborate and participate: Engage with the NLP community
by collaborating with others, participating in competitions, or contributing to open-source NLP projects. Working 
with others and sharing your work can provide valuable feedback and help you grow as an NLP 
practitioner.\n\nRemember that NLP is a vast field, and it's important to focus on specific areas of interest while
also having a good understanding of the broader concepts. Continuous learning, practice, and hands-on experience 
will help you become proficient in NLP.",
            additional_kwargs={},
          

### Saving And Extending Conversations

- Add the ability for the chatbot to remember previous conversations.
    
<br>

[![image.png](https://i.postimg.cc/NGTgxztb/image.png)](https://postimg.cc/Dmvkv58b)

<br>

> We can use:
- a JSON file.

[![image.png](https://i.postimg.cc/B6FGMXrt/image.png)](https://postimg.cc/bDqMvNNj)

In [27]:
from langchain.memory import FileChatMessageHistory

In [32]:
chat_llm = ChatOpenAI(
    model=OPENAI_CHAT_MODEL,
    temperature=TEMPERATURE,
)

memory = ConversationBufferMemory(
    chat_memory=FileChatMessageHistory(file_path="chat_history.json"),  # New!
    memory_key="messages",  # key for storing the chat history
    return_messages=True,  # returns the promp classes. e.g. HumanMsg, AIMsg, etc
)

# Used for chat completion LLMs
prompt = ChatPromptTemplate(
    input_variables=["content", "messages"],
    messages=[
        HumanMessagePromptTemplate.from_template("{content}"),
        MessagesPlaceholder(
            variable_name="messages"
        ),  # used to track the conversations
    ],
)

chain: LLMChain = LLMChain(
    llm=chat_llm,
    prompt=prompt,
    memory=memory,  # Add memory
)

In [33]:
content: str = "Hello, what is 20+5?"
result: dict[str, Any] = chain({"content": content})

# Follow up question
content: str = "subtract 10 from the result"
result: dict[str, Any] = chain({"content": content})


print(result)

{
    'content': 'subtract 10 from the result',
    'messages': [
        HumanMessage(content='Hello, what is 20+5?', additional_kwargs={}, example=False),
        AIMessage(content='20 + 5 equals 25.', additional_kwargs={}, example=False)
    ],
    'text': 'To subtract 10 from the result, you would subtract 10 from 25:\n\n25 - 10 = 15'
}